#Limpieza HTML

### Resumen del Script

Este script automatiza varias tareas de limpieza, validación y procesamiento de datos en un archivo Excel, asegurando que los datos cumplan con ciertos requisitos específicos. A continuación, se detalla lo que realiza el script:

1. **Carga del Archivo Excel**:
   - Permite al usuario subir un archivo Excel con extensión `.xlsx` o `.xls`.
   - El archivo se lee en un DataFrame para su posterior procesamiento.

2. **Limpieza de Texto**:
   - Elimina caracteres no deseados como comillas simples y dobles, así como símbolos de marca registrada y de marca comercial de las columnas "Descripción comercial" y "Nombre Comercial".

3. **Validación de Capitalización**:
   - Asegura que solo la primera letra en la columna "Recomendaciones de uso" esté en mayúscula.
   - Registra en una nueva columna si el texto cumple o no con este criterio.

4. **Validación de la Estructura HTML**:
   - Valida que el HTML en la columna "Descripción comercial" siga la estructura requerida:
     - Debe empezar con una etiqueta `<h3>` cuyo contenido coincida con el "Nombre Comercial".
     - Debe incluir al menos un párrafo `<p>`.
     - Debe contener un segundo `<h3>` con el texto "Beneficios del producto".
     - Debe contar con al menos una lista `<ul>`.

5. **Registro de Validaciones**:
   - Se crea una columna "Validación" que registra cualquier problema encontrado con caracteres no deseados en las columnas "Descripción comercial" y "Nombre Comercial".
   - La columna "Validación" también registra si la capitalización de "Recomendaciones de uso" es incorrecta.

6. **Guardado del Archivo Modificado**:
   - El DataFrame procesado se guarda en un nuevo archivo Excel, que se descarga automáticamente al final de la ejecución del script.

Este script es útil para asegurar la coherencia y calidad de los datos antes de utilizarlos en una tienda en línea u otro entorno donde la precisión del contenido es crucial.


In [1]:
!pip install pandas openpyxl


Columnas obligatorias en el archivo excel

* Nombre Comercial
* Descripción comercial

In [5]:
import pandas as pd
from google.colab import files
from bs4 import BeautifulSoup

# Subir el archivo Excel
uploaded = files.upload()

# Verificar y leer el archivo Excel subido
for filename in uploaded.keys():
    print(f'Se ha subido el archivo: {filename}')
    archivo_excel = filename

    if archivo_excel.endswith('.xlsx'):
        df = pd.read_excel(archivo_excel, engine='openpyxl')
    elif archivo_excel.endswith('.xls'):
        df = pd.read_excel(archivo_excel, engine='xlrd')
    else:
        raise ValueError('Formato de archivo no soportado. Por favor, sube un archivo .xlsx o .xls')

# Función para limpiar texto de comillas y símbolos no deseados
def limpiar_texto(texto):
    if pd.isna(texto):
        return texto
    texto_limpio = texto
    texto_limpio = texto_limpio.replace('“', '').replace('”', '')  # Eliminar comillas dobles
    texto_limpio = texto_limpio.replace('‘', '').replace('’', '')  # Eliminar comillas simples
    texto_limpio = texto_limpio.replace('™', '').replace('®', '')  # Eliminar símbolos
    return texto_limpio

# Función para verificar si un texto contiene caracteres no deseados
def verificar_validacion(texto):
    if pd.isna(texto):
        return ''
    if any(char in texto for char in ['“', '”', '‘', '’', '™', '®']):
        return 'Contiene caracteres no deseados'
    return 'Sin caracteres no deseados'

# Función para validar que solo la primera letra de la columna esté en mayúscula
def validar_mayuscula(texto):
    if pd.isna(texto):
        return ''
    texto_limpio = texto.strip()  # Eliminar espacios al inicio y al final
    if texto_limpio and texto_limpio[0].islower():
        return 'Primera letra en minúscula'
    return 'Correcto'

# Función para asegurarse de que solo la primera letra esté en mayúscula
def asegurar_primera_mayuscula(texto):
    if pd.isna(texto):
        return texto
    texto = texto.strip()  # Eliminar espacios al inicio y al final
    if texto:
        return texto[0].upper() + texto[1:].lower()
    return texto

# Función para validar la estructura del HTML
def validar_html(html, nombre_comercial):
    if pd.isna(html):
        return 'HTML vacío'
    soup = BeautifulSoup(html, 'html.parser')

    # Validar que el primer h3 coincida con el nombre comercial
    h3_tags = soup.find_all('h3')
    if not h3_tags or h3_tags[0].text.strip() != nombre_comercial.strip():
        return 'Error en el primer <h3> o no coincide con Nombre Comercial'

    # Validar que haya al menos un párrafo <p>
    p_tags = soup.find_all('p')
    if len(p_tags) < 1:
        return 'Debe haber al menos un párrafo <p>'

    # Validar que el segundo <h3> tenga el texto "Beneficios del producto"
    if len(h3_tags) < 2 or h3_tags[1].text.strip() != "Beneficios del producto":
        return 'Error en el segundo <h3> o no coincide con "Beneficios del producto"'

    # Validar que haya al menos una lista <ul>
    ul_tags = soup.find_all('ul')
    if len(ul_tags) < 1:
        return 'Debe haber al menos una lista <ul>'

    return 'Estructura HTML válida'

# Aplicar las funciones de limpieza y validación en las columnas correspondientes
df['Descripción comercial'] = df['Descripción comercial'].apply(limpiar_texto)
df['Nombre Comercial'] = df['Nombre Comercial'].apply(limpiar_texto)

# Validar la estructura del HTML en la columna "Descripción comercial"
df['Validación HTML'] = df.apply(lambda row: validar_html(row['Descripción comercial'], row['Nombre Comercial']), axis=1)

# Crear una nueva columna "Validación" y registrar las celdas con caracteres no deseados
df['Validación'] = df[['Descripción comercial', 'Nombre Comercial']].apply(
    lambda row: verificar_validacion(row['Descripción comercial']) + ' | ' + verificar_validacion(row['Nombre Comercial']),
    axis=1
)

# Aplicar la función de capitalización a la columna "Recomendaciones de uso"
df['Recomendaciones de uso'] = df['Recomendaciones de uso'].apply(asegurar_primera_mayuscula)

# Actualizar la columna "Validación" con el reporte de "Recomendaciones de uso"
df['Validación recomendaciones'] = df['Recomendaciones de uso'].apply(validar_mayuscula)
df['Validación'] = df['Validación'] + ' | ' + df['Validación recomendaciones']

# Guardar el DataFrame modificado en un nuevo archivo Excel
nombre_archivo_salida = 'archivo_modificado_con_validaciones_completas.xlsx'
df.to_excel(nombre_archivo_salida, index=False, engine='openpyxl')

# Descargar el archivo Excel modificado
files.download(nombre_archivo_salida)

# Mostrar un mensaje de confirmación
print(f"El archivo {nombre_archivo_salida} ha sido guardado y está disponible para descargar.")


Saving Chile_Plantilla_Redacción (3).xlsx to Chile_Plantilla_Redacción (3) (3).xlsx
Se ha subido el archivo: Chile_Plantilla_Redacción (3) (3).xlsx


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

El archivo archivo_modificado_con_validaciones_completas.xlsx ha sido guardado y está disponible para descargar.
